In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import scipy as sp
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold, TimeSeriesSplit
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV

## 1.

In [2]:
df = pd.read_csv('training.csv')
df

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,3,0,12/7/2009,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,...,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389
3,4,0,12/7/2009,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,...,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630
4,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72978,73010,1,12/2/2009,ADESA,2001,8,MERCURY,SABLE,GS,4D SEDAN GS,...,4836.0,5937.0,NaN,NaN,18111,30212,GA,4200.0,0,993
72979,73011,0,12/2/2009,ADESA,2007,2,CHEVROLET,MALIBU 4C,LS,4D SEDAN LS,...,10151.0,11652.0,NaN,NaN,18881,30212,GA,6200.0,0,1038
72980,73012,0,12/2/2009,ADESA,2005,4,JEEP,GRAND CHEROKEE 2WD V,Lar,4D WAGON LAREDO,...,11831.0,14402.0,NaN,NaN,18111,30212,GA,8200.0,0,1893
72981,73013,0,12/2/2009,ADESA,2006,3,CHEVROLET,IMPALA,LS,4D SEDAN LS,...,10099.0,11228.0,NaN,NaN,18881,30212,GA,7000.0,0,1974


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   RefId                              72983 non-null  int64  
 1   IsBadBuy                           72983 non-null  int64  
 2   PurchDate                          72983 non-null  object 
 3   Auction                            72983 non-null  object 
 4   VehYear                            72983 non-null  int64  
 5   VehicleAge                         72983 non-null  int64  
 6   Make                               72983 non-null  object 
 7   Model                              72983 non-null  object 
 8   Trim                               70623 non-null  object 
 9   SubModel                           72975 non-null  object 
 10  Color                              72975 non-null  object 
 11  Transmission                       72974 non-null  obj

## 2. Implement train/validation/test split function

In [4]:
df = df.sort_values(by='PurchDate', ascending=True).reset_index(drop=True)

In [5]:
def my_split_by_date_val(validation_date, test_date, data, date_col, inclusive=False):
    data[date_col] = pd.to_datetime(data[date_col])
    validation_date = pd.to_datetime(validation_date)
    test_date = pd.to_datetime(test_date)
    if validation_date >= test_date:
        raise ValueError("validation_date should be less than test_date")
    if inclusive:
        train = data[data[date_col] < validation_date]
        validation = data[(data[date_col] >= validation_date) & (data[date_col] < test_date)]
        test = data[data[date_col] >= test_date]
    else:
        train = data[data[date_col] < validation_date]
        validation = data[(data[date_col] >= validation_date) & (data[date_col] < test_date)]
        test = data[data[date_col] >= test_date] 
    return train, validation, test

In [6]:
validation_date=pd.to_datetime(df['PurchDate']).quantile(1/3)
test_date=pd.to_datetime(df['PurchDate']).quantile(2/3)
train_df, valid_df, test_df = my_split_by_date_val(validation_date,test_date,df,date_col='PurchDate',inclusive=True)
train_df['PurchDate'].max()

Timestamp('2009-09-14 00:00:00')

In [7]:
valid_df ['PurchDate'].min()

Timestamp('2009-09-15 00:00:00')

In [8]:
valid_df ['PurchDate'].max()

Timestamp('2010-05-13 00:00:00')

In [9]:
test_df['PurchDate'].min()

Timestamp('2010-05-14 00:00:00')

In [10]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols

['Auction',
 'Make',
 'Model',
 'Trim',
 'SubModel',
 'Color',
 'Transmission',
 'WheelType',
 'Nationality',
 'Size',
 'TopThreeAmericanName',
 'PRIMEUNIT',
 'AUCGUART',
 'VNST']

## 3. Preprocessing of the categorical cols.

In [11]:
encoder = ce.CountEncoder()
encoder.fit(train_df[categorical_cols])

,verbose,0
,cols,"['Auction', 'Make', ...]"
,drop_invariant,False
,return_df,True
,handle_unknown,'value'
,handle_missing,'value'
,min_group_size,None
,combine_min_nan_groups,True
,min_group_name,None
,normalize,False


In [12]:
train_df=train_df.copy()
valid_df=valid_df.copy()
test_df=test_df.copy()
train_df[categorical_cols]=encoder.transform(train_df[categorical_cols])
valid_df[categorical_cols]=encoder.transform(valid_df[categorical_cols])
test_df[categorical_cols]=encoder.transform(test_df[categorical_cols])
train_df

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
38,8899,0,2009-01-12,14038,2004,5,4766,166,424,160,...,7821.0,9413.0,24230,24230,18880,27542,2444,6900.0,0,1763
39,19212,0,2009-01-12,5224,2005,4,4250,308,3421,96,...,4577.0,5777.0,24230,24230,21053,95673,2547,4590.0,0,1506
40,8886,0,2009-01-12,14038,2005,4,284,53,232,5,...,6870.0,8754.0,24230,24230,18880,27542,2444,6100.0,0,1155
41,8887,0,2009-01-12,14038,2005,4,4766,328,3421,511,...,4031.0,4996.0,24230,24230,16044,27542,2444,3300.0,0,1918
42,8888,0,2009-01-12,14038,2003,6,319,19,2979,54,...,5615.0,6318.0,24230,24230,18880,27542,2444,5800.0,0,805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72819,27480,1,2009-09-09,5224,2003,6,4250,128,789,76,...,3444.0,4560.0,24230,24230,835,85009,2449,5385.0,0,2322
72820,46569,0,2009-09-09,14038,2006,3,2760,123,711,285,...,5496.0,6489.0,24230,24230,20928,33809,3387,4910.0,0,1272
72821,46570,0,2009-09-09,14038,2005,4,1437,682,267,166,...,7222.0,8777.0,24230,24230,20928,33809,3387,7610.0,0,1974
72822,46571,0,2009-09-09,14038,2006,3,5644,665,2979,1255,...,7683.0,8968.0,24230,24230,20928,33809,3387,8045.0,0,2152


In [13]:
X_train = train_df.drop(['PurchDate', 'IsBadBuy'], axis=1).copy()
y_train = train_df['IsBadBuy'].copy()
X_valid = valid_df.drop(['PurchDate', 'IsBadBuy'], axis=1).copy()
y_valid = valid_df['IsBadBuy'].copy()
X_train=X_train.fillna(X_train.mean())
X_valid=X_valid.fillna(X_train.mean())
X_test=test_df.drop(['PurchDate', 'IsBadBuy'], axis=1).copy()
y_test = test_df['IsBadBuy'].copy()
feature_columns = X_train.columns.tolist()
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_valid_sc = scaler.transform(X_valid)
X_test=scaler.transform(X_test)

## 4. Use LogisticRegression, GaussianNB, KNN from sklearn. Check gini on the validation set.

In [14]:
lr=LogisticRegression(random_state=21,max_iter=1000,)
lr.fit(X_train_sc,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,21
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [15]:
y_proba_lr = lr.predict_proba(X_valid_sc)[:, 1]
y_train = y_train.to_numpy()

In [16]:
def calculate_gini(y_train, y_probs):
    auc = roc_auc_score(y_train, y_probs)
    gini = 2 * auc - 1
    return gini, auc
gini_lr,auc_lr=calculate_gini(y_valid,y_proba_lr)

In [17]:
results={}
results['LogisticRegression'] = {'gini': gini_lr, 'auc': auc_lr}

In [18]:
gnb=GaussianNB()
gnb.fit(X_train_sc,y_train)

,priors,None
,var_smoothing,1e-09


In [19]:
y_proba_gnb = gnb.predict_proba(X_valid_sc)[:, 1]
gini_gnb, auc_gnb = calculate_gini(y_valid, y_proba_gnb)

In [20]:
results['GaussianNB'] = {'gini': gini_gnb, 'auc': auc_gnb}

In [21]:
results

{'LogisticRegression': {'gini': 0.46435044268580516,
  'auc': 0.7321752213429026},
 'GaussianNB': {'gini': 0.44208883798091136, 'auc': 0.7210444189904557}}

In [22]:
knn = KNeighborsClassifier()
knn.fit(X_train_sc, y_train)
y_proba_knn = knn.predict_proba(X_valid_sc)[:, 1]
gini_knn, auc_knn = calculate_gini(y_valid, y_proba_knn)
results['KNN'] = {'gini': gini_knn, 'auc': auc_knn}

In [23]:
results

{'LogisticRegression': {'gini': 0.46435044268580516,
  'auc': 0.7321752213429026},
 'GaussianNB': {'gini': 0.44208883798091136, 'auc': 0.7210444189904557},
 'KNN': {'gini': 0.33111714530845626, 'auc': 0.6655585726542281}}

### Выводы: Так как AUC (метрика отвечающая за способность модели разделять два класса) больше всего у логистической регрессии, следовательно эта модель лучше всего справляется с этой задачей. Gini говорит о том что эта модель на 46% лучше случайного угадывания. GaussianNB требует нормального распределения и независимости признаков, knn чувствителен к количеству признаков тк с их ростом расстояние между ними становится все менее информативно. 

## 5. Implement Gini and check its works

In [24]:
def my_gini(y_true, y_pred_proba):
    y_true = np.asarray(y_true)
    y_pred_proba = np.asarray(y_pred_proba)
    pos_idcs = np.where(y_true == 1)[0]
    neg_idcs = np.where(y_true == 0)[0]
    n_pos = len(pos_idcs)
    n_neg = len(neg_idcs)
    if n_pos == 0 or n_neg == 0:
        return 0.0, 0.5
    cor_pairs = 0
    eq_pairs = 0
    for i in pos_idcs:
        for j in neg_idcs:
            if y_pred_proba[i] > y_pred_proba[j]:
                cor_pairs += 1
            elif y_pred_proba[i] == y_pred_proba[j]:
                eq_pairs += 1
    auc = (cor_pairs + 0.5 * eq_pairs) / (n_pos * n_neg)
    gini = 2 * auc - 1
    return gini, auc

In [25]:
my_lr=LogisticRegression(random_state=21,max_iter=1000,class_weight='balanced')
my_lr.fit(X_train_sc,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,21
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [26]:
y_my_proba_lr = my_lr.predict_proba(X_valid_sc)[:, 1]

In [27]:
mygini_lr,myauc_lr=my_gini(y_valid,y_my_proba_lr)

In [28]:
results['LogisticRegression (my gini)'] = {'gini': mygini_lr, 'auc': myauc_lr}

In [29]:
results['LogisticRegression (my gini)']

{'gini': 0.4520894520806411, 'auc': 0.7260447260403206}

In [30]:
gnb=GaussianNB()
gnb.fit(X_train_sc,y_train)
y_my_proba_gnb = gnb.predict_proba(X_valid_sc)[:, 1]
my_gini_gnb, my_auc_gnb = my_gini(y_valid, y_my_proba_gnb)
results['GaussianNB (my gini)'] = {'gini': my_gini_gnb, 'auc': my_auc_gnb}

In [31]:
knn = KNeighborsClassifier()
knn.fit(X_train_sc, y_train)
y_my_proba_knn = knn.predict_proba(X_valid_sc)[:, 1]
my_gini_knn, my_auc_knn =my_gini(y_valid, y_my_proba_knn)
results['KNN (my gini)'] = {'gini': my_gini_knn, 'auc': my_auc_knn}

In [32]:
results

{'LogisticRegression': {'gini': 0.46435044268580516,
  'auc': 0.7321752213429026},
 'GaussianNB': {'gini': 0.44208883798091136, 'auc': 0.7210444189904557},
 'KNN': {'gini': 0.33111714530845626, 'auc': 0.6655585726542281},
 'LogisticRegression (my gini)': {'gini': 0.4520894520806411,
  'auc': 0.7260447260403206},
 'GaussianNB (my gini)': {'gini': 0.44208883798091136,
  'auc': 0.7210444189904557},
 'KNN (my gini)': {'gini': 0.33111714530845604, 'auc': 0.665558572654228}}

## 6. Implement LogisticRegression, KNN, NaiveBayes classifiers.

In [33]:
class my_logistic_regression:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.w = None
        self.b = None
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0
        for i in range(self.n_iters):
            linear = np.dot(X, self.w) + self.b
            y_pred = self._sigmoid(linear)
            dw = (1/n_samples) * np.dot(X.T, (y_pred - y))
            db = (1/n_samples) * np.sum(y_pred - y)
            self.w = self.w - self.lr * dw
            self.b = self.b - self.lr * db
    def predict_proba(self, X):
        linear = np.dot(X, self.w) + self.b
        return self._sigmoid(linear)
    def predict(self, X, threshold=0.5):
        probs = self.predict_proba(X)
        return (probs >= threshold).astype(int)

In [34]:
lr=my_logistic_regression()

In [35]:
lr.fit(X_train_sc,y_train)
y_proba_mylr = lr.predict_proba(X_valid_sc)
gini_mylr, auc_mylr = calculate_gini(y_valid, y_proba_mylr)
results['my Logistic Regression'] = {'gini': gini_mylr, 'auc': auc_mylr}

In [36]:
results['my Logistic Regression']

{'gini': 0.45882025211908917, 'auc': 0.7294101260595446}

In [37]:
results

{'LogisticRegression': {'gini': 0.46435044268580516,
  'auc': 0.7321752213429026},
 'GaussianNB': {'gini': 0.44208883798091136, 'auc': 0.7210444189904557},
 'KNN': {'gini': 0.33111714530845626, 'auc': 0.6655585726542281},
 'LogisticRegression (my gini)': {'gini': 0.4520894520806411,
  'auc': 0.7260447260403206},
 'GaussianNB (my gini)': {'gini': 0.44208883798091136,
  'auc': 0.7210444189904557},
 'KNN (my gini)': {'gini': 0.33111714530845604, 'auc': 0.665558572654228},
 'my Logistic Regression': {'gini': 0.45882025211908917,
  'auc': 0.7294101260595446}}

In [38]:
class my_KNN:
    def __init__(self, k=5):
        self.k = k
        self.X_train = None
        self.y_train = None
    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        return self 
    def predict_proba(self, X):
        X = np.array(X)
        probs = []
        for test_point in X:
            distances = np.sqrt(np.sum((self.X_train - test_point) ** 2, axis=1))
            near_idcs = np.argsort(distances)[:self.k]
            near_lbls = self.y_train[near_idcs]
            prob_class_1 = np.mean(near_lbls == 1)
            probs.append(prob_class_1)
        return np.array(probs)
    def predict(self, X, threshold=0.5):
        probs = self.predict_proba(X)
        return (probs >= threshold).astype(int)

In [39]:
knn = my_KNN(k=5)
knn.fit(X_train_sc, y_train)
y_proba_myknn = knn.predict_proba(X_valid_sc)
gini_myknn, auc_myknn = calculate_gini(y_valid, y_proba_myknn)
results['my KNN'] = {'gini': gini_myknn, 'auc': auc_myknn}
results['my KNN']

{'gini': 0.33111714530845626, 'auc': 0.6655585726542281}

In [40]:
class my_naive_Bayes:
    def __init__(self):
        self.prior_0 = None      
        self.prior_1 = None      
        self.means_0 = None      
        self.means_1 = None      
        self.vars_0 = None      
        self.vars_1 = None  
    
    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.prior_0 = np.mean(y == 0) 
        self.prior_1 = np.mean(y == 1) 
        self.means_0 = np.mean(X[y == 0], axis=0)
        self.means_1 = np.mean(X[y == 1], axis=0)
        self.vars_0 = np.var(X[y == 0], axis=0)
        self.vars_1 = np.var(X[y == 1], axis=0)
        self.vars_0 = np.maximum(self.vars_0, 1e-9)
        self.vars_1 = np.maximum(self.vars_1, 1e-9)
        return self
    
    def _gaussian_log_pdf(self, x, mean, var):
        return -0.5 * np.log(2 * np.pi * var) - ((x - mean) ** 2) / (2 * var)
    
    def predict_proba(self, X):
        X = np.array(X)
        n_samples = X.shape[0]
        n_features = X.shape[1]
        probs = []
        for i in range(n_samples):
            x = X[i]
            log_prob_0 = np.log(self.prior_0)
            log_prob_1 = np.log(self.prior_1)
            for j in range(n_features):
                log_prob_0 += self._gaussian_log_pdf(x[j], self.means_0[j], self.vars_0[j])
                log_prob_1 += self._gaussian_log_pdf(x[j], self.means_1[j], self.vars_1[j])
            max_log = max(log_prob_0, log_prob_1)
            exp_0 = np.exp(log_prob_0 - max_log)
            exp_1 = np.exp(log_prob_1 - max_log)
            total = exp_0 + exp_1
            prob_1_norm = exp_1 / total if total > 0 else 0.5
            probs.append(prob_1_norm)
        return np.array(probs)
    def predict(self, X, threshold=0.5):
        proba = self.predict_proba(X)
        return (proba >= threshold).astype(int)

In [41]:
bs = my_naive_Bayes()
bs.fit(X_train_sc, y_train)
y_proba_mybs = bs.predict_proba(X_valid_sc)
gini_mybs, auc_mybs = calculate_gini(y_valid, y_proba_mybs)
results['my Naive Bayes'] = {'gini': gini_mybs, 'auc': auc_mybs}
results['my Naive Bayes']

{'gini': 0.44208883798091136, 'auc': 0.7210444189904557}

In [42]:
results

{'LogisticRegression': {'gini': 0.46435044268580516,
  'auc': 0.7321752213429026},
 'GaussianNB': {'gini': 0.44208883798091136, 'auc': 0.7210444189904557},
 'KNN': {'gini': 0.33111714530845626, 'auc': 0.6655585726542281},
 'LogisticRegression (my gini)': {'gini': 0.4520894520806411,
  'auc': 0.7260447260403206},
 'GaussianNB (my gini)': {'gini': 0.44208883798091136,
  'auc': 0.7210444189904557},
 'KNN (my gini)': {'gini': 0.33111714530845604, 'auc': 0.665558572654228},
 'my Logistic Regression': {'gini': 0.45882025211908917,
  'auc': 0.7294101260595446},
 'my KNN': {'gini': 0.33111714530845626, 'auc': 0.6655585726542281},
 'my Naive Bayes': {'gini': 0.44208883798091136, 'auc': 0.7210444189904557}}

In [43]:
def print_results(results):
    print(f"{'Method':<30} {'Gini':<10} {'AUC':<10}")
    for model, scores in results.items():
        gini = scores['gini']
        auc = scores['auc']
        print(f"{model:<30} {gini:<10.4f} {auc:<10.4f}")
print_results(results)

Method                         Gini       AUC       
LogisticRegression             0.4644     0.7322    
GaussianNB                     0.4421     0.7210    
KNN                            0.3311     0.6656    
LogisticRegression (my gini)   0.4521     0.7260    
GaussianNB (my gini)           0.4421     0.7210    
KNN (my gini)                  0.3311     0.6656    
my Logistic Regression         0.4588     0.7294    
my KNN                         0.3311     0.6656    
my Naive Bayes                 0.4421     0.7210    


## 7. Create new features

In [44]:
new_features_train=pd.DataFrame()
new_features_valid=pd.DataFrame()
new_features_test=pd.DataFrame()

In [45]:
online_ratio_by_wheel_train = train_df.groupby('WheelTypeID')['IsOnlineSale'].mean()
online_ratio_by_wheel_valid = valid_df.groupby('WheelTypeID')['IsOnlineSale'].mean()
online_ratio_by_wheel_test = test_df.groupby('WheelTypeID')['IsOnlineSale'].mean()
new_features_train['wheel_online_ratio'] = train_df['WheelTypeID'].map(online_ratio_by_wheel_train)
new_features_train['wheel_online_deviation'] = train_df['IsOnlineSale'] - new_features_train['wheel_online_ratio']
new_features_train['wheel_online_combined'] = train_df['WheelTypeID'] * 10 + train_df['IsOnlineSale']
new_features_valid['wheel_online_ratio'] = valid_df['WheelTypeID'].map(online_ratio_by_wheel_valid)
new_features_valid['wheel_online_deviation'] = valid_df['IsOnlineSale'] - new_features_valid['wheel_online_ratio']
new_features_valid['wheel_online_combined'] = valid_df['WheelTypeID'] * 10 + valid_df['IsOnlineSale']
new_features_test['wheel_online_ratio'] = test_df['WheelTypeID'].map(online_ratio_by_wheel_test)
new_features_test['wheel_online_deviation'] = test_df['IsOnlineSale'] - new_features_test['wheel_online_ratio']
new_features_test['wheel_online_combined'] = test_df['WheelTypeID'] * 10 + test_df['IsOnlineSale']

In [46]:
train_df = pd.concat([train_df, new_features_train], axis=1)
valid_df= pd.concat([valid_df, new_features_valid], axis=1)
test_df= pd.concat([test_df, new_features_test], axis=1)

In [47]:
X_train = train_df.drop(['PurchDate', 'IsBadBuy'], axis=1).copy()
y_train = train_df['IsBadBuy'].copy()
X_valid = valid_df.drop(['PurchDate', 'IsBadBuy'], axis=1).copy()
y_valid = valid_df['IsBadBuy'].copy()
X_test = test_df.drop(['PurchDate', 'IsBadBuy'], axis=1).copy()
y_test = test_df['IsBadBuy'].copy()
X_train=X_train.fillna(X_train.mean())
X_valid=X_valid.fillna(X_valid.mean())
X_test=X_test.fillna(X_test.mean())
feature_columns = X_train.columns.tolist()
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc=scaler.transform(X_test)
X_valid_sc = scaler.transform(X_valid)

In [48]:
knn = KNeighborsClassifier()
knn.fit(X_train_sc, y_train)
y_proba_knn_newfeatures = knn.predict_proba(X_valid_sc)[:, 1]
gini_knn_newfeatures, auc_knn_newfeatures = calculate_gini(y_valid, y_proba_knn_newfeatures)
results['KNN new features'] = {'gini': gini_knn_newfeatures, 'auc': auc_knn_newfeatures}

In [49]:
results['KNN new features']

{'gini': 0.3321340143610336, 'auc': 0.6660670071805168}

In [50]:
lr=my_logistic_regression()

In [51]:
lr=LogisticRegression(random_state=21,max_iter=1000,)
lr.fit(X_train_sc,y_train)
y_proba_lr_newfeatures = lr.predict_proba(X_valid_sc)[:, 1]
gini_lr_newfeatures, auc_lr_newfeatures = calculate_gini(y_valid, y_proba_lr_newfeatures)
results['LR new features'] = {'gini': gini_lr_newfeatures, 'auc': auc_lr_newfeatures}

In [52]:
results['LR new features']

{'gini': 0.4646634852147331, 'auc': 0.7323317426073666}

In [53]:
gnb=GaussianNB()
gnb.fit(X_train_sc,y_train)
y_proba_gnb_newfeatures = gnb.predict_proba(X_valid_sc)[:, 1]
gini_gnb_newfeatures, auc_gnb_newfeatures = calculate_gini(y_valid, y_proba_gnb_newfeatures)
results['GaussianNB new features'] = {'gini': gini_gnb_newfeatures, 'auc': auc_gnb_newfeatures}

In [54]:
results['GaussianNB new features']

{'gini': 0.44460703253957257, 'auc': 0.7223035162697863}

In [55]:
print_results(results)

Method                         Gini       AUC       
LogisticRegression             0.4644     0.7322    
GaussianNB                     0.4421     0.7210    
KNN                            0.3311     0.6656    
LogisticRegression (my gini)   0.4521     0.7260    
GaussianNB (my gini)           0.4421     0.7210    
KNN (my gini)                  0.3311     0.6656    
my Logistic Regression         0.4588     0.7294    
my KNN                         0.3311     0.6656    
my Naive Bayes                 0.4421     0.7210    
KNN new features               0.3321     0.6661    
LR new features                0.4647     0.7323    
GaussianNB new features        0.4446     0.7223    


## 8. Find the most important features.Than delete useless by hand and by L1 regularization.


In [56]:
feature_imp = pd.DataFrame({
    'feature': X_train.columns,
    'coefficient': abs(lr.coef_[0])
}).sort_values('coefficient', ascending=False)
feature_imp.head(20)

,feature,coefficient
11,WheelType,0.715409
29,VehBCost,0.294870
10,WheelTypeID,0.229494
34,wheel_online_combined,0.229494
26,BYRNO,0.181277
15,TopThreeAmericanName,0.166636
2,VehYear,0.164926
5,Model,0.164436
23,MMRCurrentRetailCleanPrice,0.152292
3,VehicleAge,0.147134


In [57]:
threshold = 0.01
low_imp_features = feature_imp[feature_imp['coefficient'] < threshold]['feature'].tolist()
X_train_hand_drop = X_train.drop(columns=low_imp_features)
X_valid_hand_drop = X_valid.drop(columns=low_imp_features)

In [58]:
scaler=StandardScaler()
X_train_hand_drop_sc = scaler.fit_transform(X_train_hand_drop)
X_valid_hand_drop_sc = scaler.transform(X_valid_hand_drop)
knn = KNeighborsClassifier()
knn.fit(X_train_hand_drop_sc, y_train)
y_proba_hand_drop = knn.predict_proba(X_valid_hand_drop_sc)[:, 1]
gini_knn_hand_drop, auc_knn_hand_drop = calculate_gini(y_valid, y_proba_hand_drop)
results['KNN (hand drop)'] = {'gini': gini_knn_hand_drop, 'auc':auc_knn_hand_drop}

In [59]:
results['KNN (hand drop)']

{'gini': 0.3280588989874178, 'auc': 0.6640294494937089}

In [60]:
lr=LogisticRegression(solver='liblinear',random_state=21,max_iter=1000,)
lr.fit(X_train_hand_drop_sc,y_train)
y_proba_hand_drop = lr.predict_proba(X_valid_hand_drop_sc)[:, 1]
gini_lr_hand_drop, auc_lr_hand_drop = calculate_gini(y_valid, y_proba_hand_drop)
results['LogisticRegression (hand drop)'] = {'gini': gini_lr_hand_drop, 'auc':auc_lr_hand_drop}

In [61]:
results['LogisticRegression (hand drop)']

{'gini': 0.41526871461509884, 'auc': 0.7076343573075494}

In [62]:
gnb=GaussianNB()
gnb.fit(X_train_hand_drop_sc,y_train)
y_proba_hand_drop= gnb.predict_proba(X_valid_hand_drop_sc)[:, 1]
gini_gnb_hand_drop, auc_gnb_hand_drop = calculate_gini(y_valid, y_proba_hand_drop)
results['GaussianNB (hand drop)'] = {'gini': gini_gnb_hand_drop,'auc':auc_gnb_hand_drop}
results['GaussianNB (hand drop)']

{'gini': 0.44468615350475904, 'auc': 0.7223430767523795}

In [63]:
lg = LogisticRegression(solver='liblinear', penalty='l1', random_state=21, max_iter=1000)
lg.fit(X_train, y_train)
imp_features = X_train.columns[lg.coef_[0] != 0].tolist()
X_train_l1 = X_train[imp_features]
X_valid_l1 = X_valid[imp_features]
scaler = StandardScaler()
X_train_l1_sc = scaler.fit_transform(X_train_l1)
X_valid_l1_sc = scaler.transform(X_valid_l1)

In [64]:
lg = LogisticRegression(solver='liblinear', random_state=21, max_iter=1000)
lg.fit(X_train_l1_sc, y_train)
y_proba_l1_drop = lg.predict_proba(X_valid_l1_sc)[:, 1]
gini_lr_l1_drop, auc_lr_l1_drop = calculate_gini(y_valid, y_proba_l1_drop)
results['LogisticRegression (l1 clear)'] = {'gini': gini_lr_l1_drop,'auc':auc_lr_l1_drop}

In [65]:
results['LogisticRegression (l1 clear)']

{'gini': 0.41531695259387424, 'auc': 0.7076584762969371}

In [66]:
knn = KNeighborsClassifier()
knn.fit(X_train_l1_sc, y_train)
y_proba_l1_drop = knn.predict_proba(X_valid_l1_sc)[:, 1]
gini_knn_l1_drop, auc_knn_l1_drop = calculate_gini(y_valid, y_proba_l1_drop)
results['KNN (l1 clear)'] = {'gini': gini_knn_l1_drop,'auc':auc_knn_l1_drop}

In [67]:
results['KNN (l1 clear)']

{'gini': 0.3321340143610336, 'auc': 0.6660670071805168}

In [68]:
gnb=GaussianNB()
gnb.fit(X_train_l1_sc,y_train)
y_proba_l1_drop = gnb.predict_proba(X_valid_l1_sc)[:, 1]
gini_gnb_l1_drop,  auc_gnb_l1_drop = calculate_gini(y_valid, y_proba_l1_drop)
results['GaussianNB (l1 clear)'] = {'gini': gini_gnb_l1_drop,'auc':auc_gnb_l1_drop}
results['GaussianNB (l1 clear)']

{'gini': 0.4446068248729973, 'auc': 0.7223034124364986}

## 9. Select your best model. Fit hyperparameters to increase the Gini on the validation dataset.

In [69]:
print_results(results)

Method                         Gini       AUC       
LogisticRegression             0.4644     0.7322    
GaussianNB                     0.4421     0.7210    
KNN                            0.3311     0.6656    
LogisticRegression (my gini)   0.4521     0.7260    
GaussianNB (my gini)           0.4421     0.7210    
KNN (my gini)                  0.3311     0.6656    
my Logistic Regression         0.4588     0.7294    
my KNN                         0.3311     0.6656    
my Naive Bayes                 0.4421     0.7210    
KNN new features               0.3321     0.6661    
LR new features                0.4647     0.7323    
GaussianNB new features        0.4446     0.7223    
KNN (hand drop)                0.3281     0.6640    
LogisticRegression (hand drop) 0.4153     0.7076    
GaussianNB (hand drop)         0.4447     0.7223    
LogisticRegression (l1 clear)  0.4153     0.7077    
KNN (l1 clear)                 0.3321     0.6661    
GaussianNB (l1 clear)          0.4446     0.72

## Best model: Logistic Regression with new features

In [70]:
import optuna
def objective(trial):
    params = {
        'C': trial.suggest_float('C', 0.001, 10.0, log=True),
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet', None]),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced']),
        'tol': trial.suggest_float('tol', 0.0001, 0.01),
        'max_iter': 1000,
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'random_state': 21
    }
    penalty = params['penalty']
    if penalty == 'l1':
        params['solver'] = trial.suggest_categorical('solver_l1', ['liblinear', 'saga'])
    elif penalty == 'l2':
        params['solver'] = trial.suggest_categorical('solver_l2', ['lbfgs', 'liblinear', 'saga'])
    elif penalty == 'elasticnet':
        params['solver'] = 'saga' 
        params['l1_ratio'] = trial.suggest_float('l1_ratio', 0.1, 0.9)
    else: 
        params['solver'] = trial.suggest_categorical('solver_none', ['lbfgs', 'saga'])
    try:
        model = LogisticRegression(**params)
        model.fit(X_train_sc, y_train)
        y_proba = model.predict_proba(X_valid_sc)[:, 1]
        return 2*roc_auc_score(y_valid, y_proba)-1
    except Exception as e:
        return float('-inf') 
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2025-10-01 17:51:38,830] A new study created in memory with name: no-name-07168366-fa49-400e-8d8d-d75f1bbb9518
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1232: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
[I 2025-10-01 17:51:38,871] Trial 0 finished with value: 0.47016027112948056 and parameters: {'C': 0.0014274913077295273, 'penalty': None, 'class_weight': 'balanced', 'tol': 0.005067068723796912, 'fit_intercept': False, 'solver_none': 'lbfgs'}. Best is trial 0 with value: 0.47016027112948056.
[I 2025-10-01 17:51:38,976] Trial 1 finished with value: 0.46549274234986004 and parameters: {'C': 0.0010734807383973482, 'penalty': 'l1', 'class_weight': 'balanced', 'tol': 0.004473713298640451, 'fit_intercept': True, 'solver_l1': 'saga'}. Best is trial 0 with value: 0.47016027112948056.
[I 2025-10-01 17:51:39,097] Trial 2 finished with value: 0.4700900204937244 and parameters: {'C': 0.12735884800776118, 'penal

In [71]:
print(f"best parameters for gini: {study.best_params}")
print(f"best gini: {study.best_value}")

best parameters for gini: {'C': 0.0014274913077295273, 'penalty': None, 'class_weight': 'balanced', 'tol': 0.005067068723796912, 'fit_intercept': False, 'solver_none': 'lbfgs'}
best gini: 0.47016027112948056


In [72]:
best_model = LogisticRegression(C=study.best_params['C'],penalty='l1', solver='liblinear',   class_weight=study.best_params['class_weight'],tol=study.best_params['tol'],fit_intercept=study.best_params['fit_intercept'],random_state=21,max_iter=1000)
best_model.fit(X_train_sc, y_train)
y_proba_train = best_model.predict_proba(X_train_sc)[:, 1]
y_proba_valid = best_model.predict_proba(X_valid_sc)[:, 1]
y_proba_test=best_model.predict_proba(X_test_sc)[:,1]
gini_train = 2*roc_auc_score(y_train, y_proba_train)-1
gini_valid = 2*roc_auc_score(y_valid, y_proba_valid)-1
gini_test = 2*roc_auc_score(y_test, y_proba_test)-1

In [73]:
gini_train

0.4757933261606926

In [74]:
gini_valid

0.46585630718989135

In [75]:
gini_test

0.47795515663517807

In [76]:
print(f"Train: {y_train.mean():.3f}")
print(f"Valid: {y_valid.mean():.3f}") 
print(f"Test: {y_test.mean():.3f}")

Train: 0.115
Valid: 0.130
Test: 0.124


### Подобрав параметры с помощью модуля optuna удалось добиться увеличения gini. Самые важные параметры для logistic regression - C(параметр регулизации), penalty (тип регулизации), solver(алгоритм оптимизации), class_weight (балансировка классов)

### Наблюдается переобучение модели за счет разного распределения целеыой переменной по временным промежуткам. Споcобы решения: временная кросс-валидация, более сильная регулизация, использовать модели менее чувствительные к сдвигу модели.

## 10. Implement metrics and compare with sklearn

In [77]:
def my_recall(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    if (tp + fn) == 0:
        return 0.0
    return tp / (tp + fn)

In [78]:
def my_precision(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    if (tp + fp) == 0:
        return 0.0
    return tp / (tp + fp)

In [79]:
def my_f1(y_true, y_pred):
    precision = my_precision(y_true, y_pred)
    recall = my_recall(y_true, y_pred)
    if (precision + recall) == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

In [80]:
best_model = LogisticRegression(C=study.best_params['C'],penalty='l1', solver='liblinear',   class_weight=study.best_params['class_weight'],tol=study.best_params['tol'],fit_intercept=study.best_params['fit_intercept'],random_state=21,max_iter=1000)
best_model.fit(X_train_sc, y_train)
y_pred_test = best_model.predict(X_test_sc)
y_proba_test = best_model.predict_proba(X_test_sc)[:, 1]

In [81]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve, auc

In [82]:
def my_auc_pr(y_true, y_proba):
    thresholds = np.sort(np.unique(y_proba))[::-1] 
    precisions = []
    recalls = []
    for threshold in thresholds:
        y_pred = (y_proba >= threshold).astype(int)
        precision = my_precision(y_true,y_pred)
        recall = my_recall(y_true,y_pred)
        precisions.append(precision)
        recalls.append(recall)
    recalls = np.array([0.0] + recalls + [1.0])
    precisions = np.array([1.0] + precisions + [precisions[-1] if precisions else 0])
    sort_idx = np.argsort(recalls)
    recalls_sorted = recalls[sort_idx]
    precisions_sorted = precisions[sort_idx]
    auc_pr = np.trapz(precisions_sorted, recalls_sorted)
    return auc_pr

In [83]:
print(f'My metrics: Precision: {my_precision(y_test, y_pred_test)}, Recall: {my_recall(y_test, y_pred_test)}, F1: {my_f1(y_test, y_pred_test)}')
print(f'Sklearn metrics: Precision:{ precision_score(y_test, y_pred_test)}, Recall: {recall_score(y_test, y_pred_test)}, F1: {f1_score(y_test, y_pred_test)}')

My metrics: Precision: 0.2573151007204823, Recall: 0.5796621397813846, F1: 0.3564154786150713
Sklearn metrics: Precision:0.2573151007204823, Recall: 0.5796621397813846, F1: 0.3564154786150713


In [84]:
my_auc=my_auc_pr(y_test,y_proba_test)
precision, recall, thresholds = precision_recall_curve(y_test, y_proba_test)
auc_pr = auc(recall, precision)

In [85]:
my_auc

0.4130760027286595

In [86]:
auc_pr

0.4130954709853634

In [87]:
knn = KNeighborsClassifier()
knn.fit(X_train_sc, y_train)
y_proba_test = knn.predict_proba(X_test_sc)[:, 1]
my_auc=my_auc_pr(y_test,y_proba_test)
precision, recall, thresholds = precision_recall_curve(y_test, y_proba_test)
auc_pr = auc(recall, precision)

In [88]:
my_auc

0.340379617512848

In [89]:
auc_pr

0.340379617512848

In [90]:
gnb=GaussianNB()
gnb.fit(X_train_sc,y_train)
y_proba_test = gnb.predict_proba(X_test_sc)[:, 1]
my_auc=my_auc_pr(y_test,y_proba_test)
precision, recall, thresholds = precision_recall_curve(y_test, y_proba_test)
auc_pr = auc(recall, precision)

In [91]:
my_auc

0.1836378348000518

In [92]:
auc_pr

0.1836425709620894

### Таким образом для определения некачественных автомобилей для покупки стоит использовать метрики: Precision, Recall, F1-score и AUC-PR. Они образуют комплексную систему оценки, где Precision показывает точность выявления "лимонов" (минимизируя ложные обвинения хороших машин), Recall гарантирует максимальное покрытие проблемных автомобилей, F1-score обеспечивает баланс между этими целями, а AUC-PR оценивает общее качество модели на несбалансированных данных.

